In [ ]:
import requests 
!pip install bs4
!pip install html5lib
from bs4 import BeautifulSoup


In [ ]:
text_reviews = []
type_of_traveler = []
aircraft = []
seat_type = []
seat_comfort_rating = []
cabin_staff_service_rating = []
food_and_beverages_rating = []
inflight_entertainment_rating = []
ground_service_rating = []
value_for_money_rating = []
recommended = []

for i in range(1, 375):
    url = f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=10"
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html5lib')
    for para in soup.find_all("div", {"class": "text_content"}):
        text_reviews.append(para.get_text())
  
    for review_stats_elem in soup.find_all("div", {"class": "review-stats"}):
       
        
         
        if review_stats_elem:
            # Info about the passengers and fleet
            type_of_traveler.append(review_stats_elem.find("td", {"class": "type_of_traveller"}).find_next("td", {"class": "review-value"}).text if review_stats_elem.find("td", {"class": "type_of_traveller"}) else None)
            aircraft.append(review_stats_elem.find("td", {"class": "aircraft"}).find_next("td", {"class": "review-value"}).text if review_stats_elem.find("td", {"class": "aircraft"}) else None)
            seat_type.append(review_stats_elem.find("td", {"class": "cabin_flown"}).find_next("td", {"class": "review-value"}).text if review_stats_elem.find("td", {"class": "cabin_flown"}) else None)
            print("working here")
            
            # rating given by passengers in 6 different categories out of 5 
            for rating_category in ["seat_comfort", "cabin_staff_service", "food_and_beverages", "inflight_entertainment", "ground_service", "value_for_money"]:
                rating_elem = review_stats_elem.find("td", {"class": rating_category})
                if rating_elem:
                    rating_stars_elem = rating_elem.find_next("td", {"class": "review-rating-stars"})
                    if rating_stars_elem:
                        rating_stars = rating_stars_elem.find_all("span", {"class": "star fill"})
                        rating_value = len(rating_stars)
                        print("working here")
                        globals()[f"{rating_category}_rating"].append(rating_value)
                    else:
                        print("error")
                        globals()[f"{rating_category}_rating"].append(0)
                else:
                    print("error2")
                    globals()[f"{rating_category}_rating"].append(0)

 
            # extracting recommendation in yes or no 
            recommended_elem = review_stats_elem.find("td", {"class": "recommended"})
            next_elem = recommended_elem.find_next("td", {"class": "review-value"})

            if next_elem:
               recommended_value = next_elem.text.strip().lower()
               if recommended_value == "yes":
                recommended.append("yes")
               elif recommended_value == "no":
                recommended.append("no")
               else:
                recommended.append("NONE") 
            else:
               recommended.append("NONE")  

        print(f"Processed {len(text_reviews)} reviews")

print(f"Total scraped reviews: {len(text_reviews)}")



In [ ]:
import pandas as pd
#making data frame

data = {
    'Text Review': text_reviews,
    'Type of Traveler': type_of_traveler,
    'Aircraft': aircraft,
    'Seat Type': seat_type,
    'Seat Comfort Rating': seat_comfort_rating,
    'Cabin Staff Service Rating': cabin_staff_service_rating,
    'Food Beverages Rating': food_and_beverages_rating,
    'Inflight Entertainment Rating': inflight_entertainment_rating,
    'Ground Service Rating': ground_service_rating,
    'Value for Money Rating': value_for_money_rating,
    'Recommended': recommended
}

df = pd.DataFrame(data)
df.head(30)



In [ ]:
df.info()

In [ ]:
df= df.fillna(0)
df.info()
df['Text Review']=df['Text Review'].str.replace('✅ Trip Verified |','').str.replace('Not Verified | ','')
df.head()

In [8]:
df.to_csv('../DATA/BA_reviews.csv')
